In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## loading dataset

In [ ]:
train=pd.read_csv('../input/titanic/train.csv') # reading train data
test=pd.read_csv('../input/titanic/test.csv') # reading test data
train

In [ ]:
train.head() # to print first 5 rows of the data set

In [ ]:
train.shape # to print number of rows and columns

In [ ]:
train.columns # to get the column names

In [ ]:
train.describe() 

In [ ]:
train.isnull().sum() # number of null values in dataset

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
train.Sex.value_counts()

In [ ]:
train.groupby('Sex').Survived.value_counts()

In [ ]:
sns.barplot(x='Sex', y='Survived', data=train)

In [ ]:
train['Embarked'].value_counts()

In [ ]:
train.groupby('Embarked').Survived.value_counts()

In [ ]:
sns.countplot(x='Embarked', data=train)

In [ ]:
train['Pclass'].value_counts()

In [ ]:
sns.countplot(x='Pclass', data=train)

In [ ]:
train['SibSp'].value_counts()

In [ ]:
train.groupby('SibSp').Survived.value_counts()

In [ ]:
sns.countplot(x='SibSp', data=train)

In [ ]:
sns.countplot(x='Survived',hue='Pclass',data=train) # survival based on passenger class

In [ ]:
sns.distplot(train['Age'].dropna(),kde=False,bins=30)

In [ ]:
sns.countplot(x='SibSp',data=train) # count based on sibling or spouse

In [ ]:
train['Fare'].hist(bins=40,figsize=(10,4))

In [ ]:
sns.countplot(x='Parch',data=train)

In [ ]:
train['Died'] = 1 - train['Survived']

In [ ]:
train.groupby('Sex').agg('mean')[['Survived', 'Died']].plot(kind='bar', figsize=(10, 5),stacked=True)

Women are more likely to survive.

In [ ]:
sns.barplot(x='Fare', y='Survived', data=train)

In [ ]:
sns.barplot(x='SibSp',y='Survived',data=train)

## feature extraction

In [ ]:
titles=set()
for name in train['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())
print(titles)

In [ ]:
titles_dict={'Mrs':'Mrs','Major':'Other','Master':'Master','Lady':'Other','Mlle':'Miss','Dr':'Other','Col':'Other','Capt':'Other','Don':'Other','the Countess':'Other','Mme':'Mrs','Miss':'Miss','Jonkheer':'Other','Rev':'Other','Sir':'Other','Ms':'Miss','Mr':'Mr'}


In [ ]:
train['Title'] = train['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
train['Title'] = train.Title.map(titles_dict)
train.head()

In [ ]:
df1=train.drop(['Name','Ticket','Cabin','PassengerId','Died'], axis=1)
df1.head()

In [ ]:
df1.Sex=df1.Sex.map({'female':0, 'male':1})
df1.Title=df1.Title.map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5})
df1.Embarked=df1.Embarked.map({'S':0, 'C':1, 'Q':2,'nan':'NaN'})

In [ ]:
df1

In [ ]:
train.Title.value_counts()

In [ ]:
mean_men=df1[df1['Sex']==0]['Age'].mean()
mean_female=df1[df1['Sex']==1]['Age'].mean()

In [ ]:
df1.loc[(df1.Age.isnull())&(df1['Sex']==1),'Age']=mean_female
df1.loc[(df1.Age.isnull())&(df1['Sex']==0),'Age']=mean_men

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1.isnull().sum()

## feature scaling

In [ ]:
df1.Age=(df1.Age-min(df1.Age))/(max(df1.Age)-min(df1.Age))
df1.Fare=(df1.Fare-min(df1.Fare))/(max(df1.Fare)-min(df1.Fare))

In [ ]:
df1.head()

## Data modelling

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df1.drop(['Survived'],axis=1),df1.Survived,test_size=0.25,random_state=0,stratify=df1.Survived)

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf= LogisticRegression()
clf.fit(x_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred= clf.predict(x_test)
accuracy_score (y_test,y_pred)


In [ ]:
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_test, y_pred)
matrix

In [ ]:
sns.heatmap(matrix,annot=True)

## Cleaning test dataset

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
test.head()

In [ ]:
titles= set()
for name in test['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())
print(titles)

In [ ]:
titles_dict

In [ ]:
test['Title'] = test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
test['Title'] = test.Title.map(titles_dict)
test.head()

In [ ]:
df2=test.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)

In [ ]:
df2.Sex=df2.Sex.map({'female':0, 'male':1})
df2.Embarked=df2.Embarked.map({'S':0, 'C':1, 'Q':2,'nan':'Nan'})
df2.Title=df2.Title.map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5})


In [ ]:
df2.head()

In [ ]:
df2.isnull().sum()

In [ ]:
mean_female=df2[df2['Sex']==1]['Age'].mean()
mean_men=df2[df2['Sex']==0]['Age'].mean()

In [ ]:
df2.loc[(df2.Age.isnull())&(df2['Sex']==1),'Age']=mean_female
df2.loc[(df2.Age.isnull())&(df2['Sex']==0),'Age']=mean_men

In [ ]:
df2.isnull().sum()

In [ ]:
df2['Fare']=df2['Fare'].fillna(df2['Fare'].mean())

In [ ]:
df2.isnull().sum()

In [ ]:
df2[df2.Title.isnull()]

In [ ]:
df2['Title']=df2.Title.fillna(3)


In [ ]:
df2.isnull().sum()

In [ ]:
df2.head()

In [ ]:
df2['Age']=(df2.Age-min(df2.Age))/(max(df2.Age)-min(df2.Age))

In [ ]:
df2['Fare']=(df2.Fare-min(df2.Fare))/(max(df2.Fare)-min(df2.Fare))

In [ ]:
df2.head()

## Prediction

In [ ]:
pred=clf.predict(df2)

In [ ]:
pred

In [ ]:
submit=pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':pred})
submit.to_csv('./submission.csv',index=False)

In [ ]:
pred_df = pd.read_csv('./submission.csv')

In [ ]:
sns.countplot(x='Survived', data=pred_df)